In [3]:
import pandas as pd
from hdfs import InsecureClient

In [2]:
road = pd.read_csv('Road_accident.csv', delimiter=',')
date = pd.read_csv('Dates.csv', delimiter=',')

In [47]:
road.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307973 entries, 0 to 307972
Data columns (total 21 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Accident_Index              307973 non-null  object 
 1   Accident Date               307973 non-null  object 
 2   Day_of_Week                 307973 non-null  object 
 3   Junction_Control            307973 non-null  object 
 4   Junction_Detail             307973 non-null  object 
 5   Accident_Severity           307973 non-null  object 
 6   Latitude                    307973 non-null  float64
 7   Light_Conditions            307973 non-null  object 
 8   Local_Authority_(District)  307973 non-null  object 
 9   Carriageway_Hazards         5424 non-null    object 
 10  Longitude                   307973 non-null  float64
 11  Number_of_Casualties        307973 non-null  int64  
 12  Number_of_Vehicles          307973 non-null  int64  
 13  Police_Force  

In [48]:
road.head()

,Accident_Index,Accident Date,Day_of_Week,Junction_Control,Junction_Detail,Accident_Severity,Latitude,Light_Conditions,Local_Authority_(District),Carriageway_Hazards,...,Number_of_Casualties,Number_of_Vehicles,Police_Force,Road_Surface_Conditions,Road_Type,Speed_limit,Time,Urban_or_Rural_Area,Weather_Conditions,Vehicle_Type
0,200901BS70001,1/1/2021,Thursday,Give way or uncontrolled,T or staggered junction,Serious,51.512273,Daylight,Kensington and Chelsea,NaN,...,1,2,Metropolitan Police,Dry,One way street,30,15:11,Urban,Fine no high winds,Car
1,200901BS70002,1/5/2021,Monday,Give way or uncontrolled,Crossroads,Serious,51.514399,Daylight,Kensington and Chelsea,NaN,...,11,2,Metropolitan Police,Wet or damp,Single carriageway,30,10:59,Urban,Fine no high winds,Taxi/Private hire car
2,200901BS70003,1/4/2021,Sunday,Give way or uncontrolled,T or staggered junction,Slight,51.486668,Daylight,Kensington and Chelsea,NaN,...,1,2,Metropolitan Police,Dry,Single carriageway,30,14:19,Urban,Fine no high winds,Taxi/Private hire car
3,200901BS70004,1/5/2021,Monday,Auto traffic signal,T or staggered junction,Serious,51.507804,Daylight,Kensington and Chelsea,NaN,...,1,2,Metropolitan Police,Frost or ice,Single carriageway,30,8:10,Urban,Other,Motorcycle over 500cc
4,200901BS70005,1/6/2021,Tuesday,Auto traffic signal,Crossroads,Serious,51.482076,Darkness - lights lit,Kensington and Chelsea,NaN,...,1,2,Metropolitan Police,Dry,Single carriageway,30,17:25,Urban,Fine no high winds,Car


In [49]:
road['Accident Date'] = pd.to_datetime(road['Accident Date'], format='%m/%d/%Y')
road['Carriageway_Hazards'] = road['Carriageway_Hazards'].fillna('No')
road['Latitude'] = road['Latitude'].round(2)
road['Longitude'] = road['Longitude'].round(2)

In [59]:
date.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1096 entries, 0 to 1095
Data columns (total 13 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   Sl. No                              1096 non-null   int64 
 1   Name of Incident                    1096 non-null   object
 2   Date                                1096 non-null   int32 
 3   Month                               1096 non-null   object
 4   Year                                1096 non-null   int32 
 5   Era                                 1096 non-null   object
 6   Country                             1096 non-null   object
 7   Type of Event                       1096 non-null   object
 8   Place Name                          1096 non-null   object
 9   Impact                              1096 non-null   object
 10  Affected Population                 1096 non-null   object
 11  Important Person/Group Responsible  1096 non-null   obje

In [56]:
date.head()

,Sl. No,Name of Incident,Date,Month,Year,Era,Country,Type of Event,Place Name,Impact,Affected Population,Important Person/Group Responsible,Outcome
0,1,Indus Valley Civilization Flourishes,0,Unknown,2600,BC,India,Civilization,Indus Valley,Development of one of the world's earliest urb...,Local inhabitants,Indus Valley people,Positive
1,2,Battle of the Ten Kings,0,Unknown,1400,BC,India,Battle,Punjab,Rigvedic tribes consolidated their control ove...,Rigvedic tribes,Sudas,Positive
2,6,Establishment of the Delhi Sultanate,0,Unknown,1206,AD,India,Political,Delhi,Muslim rule established in parts of India,People of Delhi and surrounding regions,QutbUnknownudUnknowndin Aibak,Mixed
3,7,Battle of Panipat,21,April,1526,AD,India,Battle,Panipat,Foundation of the Mughal Empire in India,Northern Indian kingdoms,Babur,Mixed
4,8,Establishment of British Raj,1,May,1858,AD,India,Colonial,Whole India,Start of direct British governance in India,Indian subcontinent,British East India Company/Empire,Negative


In [52]:
date['Date'] = date['Date'].replace('Unknown', pd.NA)
date['Date'] = pd.to_numeric(date['Date'], errors='coerce').fillna(0).astype(int)
date.insert(date.columns.get_loc('Year') + 1, 'Era', date['Year'].apply(lambda x: 'BC' if 'BC' in x else 'AD'))
date['Year'] = date['Year'].str.replace('BC', '')
date['Year'] = date['Year'].astype(int)

In [60]:
road.to_parquet('road_accident.parquet.gzip',compression='gzip')
date.to_parquet('world_important_dates.parquet.gzip',compression='gzip')

In [27]:
client = InsecureClient('http://127.0.0.1:9870', user='root')

In [28]:
client.content('/')

{'directoryCount': 7,
 'fileCount': 7,
 'length': 91,
 'quota': 9223372036854775807,
 'spaceConsumed': 273,
 'spaceQuota': -1,
 'typeQuota': {}}

In [17]:
client.status('/sar.ius')

{'accessTime': 0,
 'blockSize': 0,
 'childrenNum': 0,
 'fileId': 16419,
 'group': 'supergroup',
 'length': 0,
 'modificationTime': 1708668475338,
 'owner': 'root',
 'pathSuffix': '',
 'permission': '755',
 'replication': 0,
 'storagePolicy': 0,
 'type': 'DIRECTORY'}

In [18]:
client.makedirs('/sar.ius')

In [19]:
client.delete('/sar.ius', recursive=True)

True

In [20]:
client.list('/')

['rmstate']

In [21]:
hdfs_path = '/sar.ius'
local_dir = 'c:\\Users\\User\\Desktop\\норм'

In [22]:
with open(local_dir + '\\road_accident.parquet.gzip') as reader, client.write(hdfs_path) as writer:
  for line in reader:
    if line.startswith('-'):
      writer.write(line)

ConnectionError: HTTPConnectionPool(host='6c0c6a2c2035', port=9864): Max retries exceeded with url: /webhdfs/v1/sar.ius?op=CREATE&user.name=root&namenoderpcaddress=namenode:9000&createflag=&createparent=true&overwrite=false&user.name=root (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x000001C129847ED0>: Failed to resolve '6c0c6a2c2035' ([Errno 11001] getaddrinfo failed)"))

In [ ]:
client.download(hdfs_path + '/road_accident_data.parquet.gzip', local_dir + '\data', overwrite=True, n_threads=1)

In [14]:
with open(local_dir + '\\world_important_dates.parquet.gzip') as reader, client.write(hdfs_path) as writer:
  for line in reader:
    if line.startswith('-'):
      writer.write(line)

ConnectionError: HTTPConnectionPool(host='6c0c6a2c2035', port=9864): Max retries exceeded with url: /webhdfs/v1/sar.ius?op=CREATE&user.name=root&namenoderpcaddress=namenode:9000&createflag=&createparent=true&overwrite=false&user.name=root (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x000001C1297C4A50>: Failed to resolve '6c0c6a2c2035' ([Errno 11001] getaddrinfo failed)"))

In [ ]:
client.download(hdfs_path + '/world_important_dates.parquet.gzip', local_dir + '\data', overwrite=True, n_threads=1)